In [1]:
import pandas as pd
from methods.make_single_df_from_bid_ask import make_single_df_from_bid_ask
# from methods.make_5min_volume_from_1_min_volume import make_5min_volume_from_1min_volume
# from machine_learning_models.lstm.eurusd.lstm_model_stats import predict_max_stats, predict_min_stats
# import torch

In [2]:
# Display all rows
pd.set_option('display.max_rows', None)

# Display all columns
pd.set_option('display.max_columns', None)

# Prevent column width truncation
pd.set_option('display.max_colwidth', None)

# Display full content without wrapping
pd.set_option('display.expand_frame_repr', False)

In [3]:
BASE_DIR = 'hist_data/'
TIME_SERIES_FOLDER = 'EURUSD/'
BID_FOLDER = '5_min/Bid/'
ASK_FOLDER = '5_min/Ask/'

BASE_DIR_LSTM = 'machine_learning_models/lstm/eurusd/pickle_files'
LSTM_MAX_VAL = BASE_DIR_LSTM + '/lstm_regressor_predict_max_values-2025-04-07.pkl'
LSTM_MIN_VAL = BASE_DIR_LSTM + '/lstm_regressor_predict_min_values-2025-04-07.pkl'
SCALER_X_MIN = BASE_DIR_LSTM + '/lstm_regressor_scaler_x_min.pkl'
SCALER_X_MAX = BASE_DIR_LSTM + '/lstm_regressor_scaler_x_max.pkl'
SCALER_Y_MIN = BASE_DIR_LSTM + '/lstm_regressor_scaler_y_min.pkl'
SCALER_Y_MAX = BASE_DIR_LSTM + '/lstm_regressor_scaler_y_max.pkl'

In [11]:
df_joined_train, df_joined_test, df_joined_val = make_single_df_from_bid_ask(
    base_dir=BASE_DIR,
    time_series_folder=TIME_SERIES_FOLDER
)

In [12]:
df_joined_train.tail()

,+DI,-DI,ADL,ADX,ATR_14,MACD,MACD_hist,MACD_signal,RSI,close,close_audusd,close_brentusd,close_cadusd,close_jpyusd,close_plus_10min,close_plus_15min,close_plus_20min,close_plus_25min,close_plus_30min,close_plus_35min,close_plus_40min,close_plus_5min,high,high_audusd,high_brentusd,high_cadusd,high_jpyusd,high_plus_10min,high_plus_15min,high_plus_20min,high_plus_25min,high_plus_30min,high_plus_35min,high_plus_40min,high_plus_5min,low,low_audusd,low_brentusd,low_cadusd,low_jpyusd,low_plus_10min,low_plus_15min,low_plus_20min,low_plus_25min,low_plus_30min,low_plus_35min,low_plus_40min,low_plus_5min,open,open_audusd,open_brentusd,open_cadusd,open_jpyusd,open_plus_10min,open_plus_15min,open_plus_20min,open_plus_25min,open_plus_30min,open_plus_35min,open_plus_40min,open_plus_5min,volume,volume_audusd,volume_brentusd,volume_cadusd,volume_jpyusd
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-11-06 21:35:00,92.366902,2.758178,-71411.888927,84.169473,0.000827,0.003462,-0.000252,0.003714,93.226069,1.07169,0.64871,85.172,1.37020,150.046,1.07180,1.07170,1.07168,1.07045,1.07059,1.07075,1.07076,1.07168,1.07183,0.64878,85.187,1.37020,150.065,1.07184,1.07184,1.07183,1.07084,1.07068,1.07077,1.07079,1.07176,1.07169,0.64871,85.142,1.36997,150.044,1.07167,1.07170,1.07161,1.07041,1.07045,1.07053,1.07061,1.07167,1.07182,0.64875,85.157,1.36997,150.062,1.07168,1.07180,1.07174,1.07079,1.07045,1.07053,1.07075,1.07171,0.000188,0.000078,3.000000e-10,27.200027,0.000599
2023-11-06 21:40:00,92.071611,2.877238,-71411.889272,88.094802,0.000766,0.002967,-0.000598,0.003565,93.113419,1.07168,0.64880,85.192,1.37007,150.061,1.07170,1.07168,1.07045,1.07059,1.07075,1.07076,1.07070,1.07180,1.07176,0.64881,85.192,1.37024,150.062,1.07184,1.07183,1.07084,1.07068,1.07077,1.07079,1.07082,1.07184,1.07167,0.64873,85.162,1.37006,150.043,1.07170,1.07161,1.07041,1.07045,1.07053,1.07061,1.07070,1.07167,1.07171,0.64873,85.162,1.37019,150.048,1.07180,1.07174,1.07079,1.07045,1.07053,1.07075,1.07076,1.07168,0.000443,0.000029,0.000000e+00,32.670035,0.000859
2023-11-06 21:45:00,91.994917,2.668361,-71411.888918,91.228607,0.000716,0.002563,-0.000802,0.003365,93.220658,1.07180,0.64887,85.172,1.37023,150.043,1.07168,1.07045,1.07059,1.07075,1.07076,1.07070,1.07053,1.07170,1.07184,0.64889,85.192,1.37026,150.061,1.07183,1.07084,1.07068,1.07077,1.07079,1.07082,1.07076,1.07184,1.07167,0.64880,85.152,1.36992,150.041,1.07161,1.07041,1.07045,1.07053,1.07061,1.07070,1.07052,1.07170,1.07168,0.64881,85.187,1.37006,150.060,1.07174,1.07079,1.07045,1.07053,1.07075,1.07076,1.07075,1.07180,0.000667,0.000087,0.000000e+00,214.960176,0.000800
2023-11-06 21:50:00,91.587603,2.593295,-71411.889512,94.449339,0.000668,0.002175,-0.000952,0.003127,91.919395,1.07170,0.64892,85.192,1.37005,150.060,1.07045,1.07059,1.07075,1.07076,1.07070,1.07053,1.07069,1.07168,1.07184,0.64892,85.202,1.37023,150.061,1.07084,1.07068,1.07077,1.07079,1.07082,1.07076,1.07088,1.07183,1.07170,0.64885,85.162,1.37003,150.042,1.07041,1.07045,1.07053,1.07061,1.07070,1.07052,1.07056,1.07161,1.07180,0.64886,85.162,1.37021,150.044,1.07079,1.07045,1.07053,1.07075,1.07076,1.07075,1.07056,1.07174,0.000594,0.000036,0.000000e+00,128.550099,0.000771
2023-11-06 21:55:00,90.214646,3.156566,-71411.889921,95.418463,0.000631,0.001839,-0.001030,0.002869,91.640296,1.07168,0.64903,85.187,1.36979,150.086,1.07059,1.07075,1.07076,1.07070,1.07053,1.07069,1.07088,1.07045,1.07183,0.64903,85.187,1.37022,150.086,1.07068,1.07077,1.07079,1.07082,1.07076,1.07088,1.07097,1.07084,1.07161,0.64892,85.162,1.36979,150.048,1.07045,1.07053,1.07061,1.07070,1.07052,1.07056,1.07068,1.07041,1.07174,0.64893,85.182,1.37019,150.051,1.07045,1.07053,1.07075,1.07076,1.07075,1.07056,1.07069,1.07079,0.001124,0.000389,0.000000e+00,341.610319,0.001214


In [13]:
df_joined_test.tail()

,+DI,-DI,ADL,ADX,ATR_14,MACD,MACD_hist,MACD_signal,RSI,close,close_audusd,close_brentusd,close_cadusd,close_jpyusd,close_plus_10min,close_plus_15min,close_plus_20min,close_plus_25min,close_plus_30min,close_plus_35min,close_plus_40min,close_plus_5min,high,high_audusd,high_brentusd,high_cadusd,high_jpyusd,high_plus_10min,high_plus_15min,high_plus_20min,high_plus_25min,high_plus_30min,high_plus_35min,high_plus_40min,high_plus_5min,low,low_audusd,low_brentusd,low_cadusd,low_jpyusd,low_plus_10min,low_plus_15min,low_plus_20min,low_plus_25min,low_plus_30min,low_plus_35min,low_plus_40min,low_plus_5min,open,open_audusd,open_brentusd,open_cadusd,open_jpyusd,open_plus_10min,open_plus_15min,open_plus_20min,open_plus_25min,open_plus_30min,open_plus_35min,open_plus_40min,open_plus_5min,volume,volume_audusd,volume_brentusd,volume_cadusd,volume_jpyusd
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-05-22 12:10:00,9.854604,53.473344,-156086.519519,24.120593,0.000479,-0.001067,-6.166149e-04,-0.000450,21.755597,1.08282,0.66488,82.058,1.36745,156.564,1.08279,1.08306,1.08313,1.08320,1.08323,1.08362,1.08345,1.08286,1.08296,0.66495,82.113,1.36767,156.577,1.08292,1.08307,1.08319,1.08328,1.08336,1.08369,1.08369,1.08305,1.08264,0.66473,81.987,1.36743,156.540,1.08272,1.08270,1.08288,1.08305,1.08313,1.08321,1.08338,1.08277,1.08295,0.66494,81.987,1.36749,156.548,1.08285,1.08280,1.08303,1.08310,1.08318,1.08321,1.08362,1.08283,1212.301206,353.43,0.0859,0.000743,548.460827
2024-05-22 12:15:00,11.272142,50.563607,-156703.462737,29.103232,0.000463,-0.001052,-4.810554e-04,-0.000570,22.578764,1.08286,0.66481,81.938,1.36753,156.587,1.08306,1.08313,1.08320,1.08323,1.08362,1.08345,1.08318,1.08279,1.08305,0.66496,82.063,1.36767,156.588,1.08307,1.08319,1.08328,1.08336,1.08369,1.08369,1.08347,1.08292,1.08277,0.66477,81.912,1.36742,156.545,1.08270,1.08288,1.08305,1.08313,1.08321,1.08338,1.08314,1.08272,1.08283,0.66487,82.063,1.36745,156.558,1.08280,1.08303,1.08310,1.08318,1.08321,1.08362,1.08347,1.08285,1727.441009,624.60,0.0597,0.000517,1701.412507
2024-05-22 12:20:00,11.326861,50.000000,-157069.015928,33.732084,0.000441,-0.000996,-3.406482e-04,-0.000656,22.134209,1.08279,0.66484,81.963,1.36737,156.584,1.08313,1.08320,1.08323,1.08362,1.08345,1.08318,1.08438,1.08306,1.08292,0.66501,81.973,1.36764,156.605,1.08319,1.08328,1.08336,1.08369,1.08369,1.08347,1.08441,1.08307,1.08272,0.66468,81.887,1.36726,156.578,1.08288,1.08305,1.08313,1.08321,1.08338,1.08314,1.08420,1.08270,1.08285,0.66482,81.933,1.36753,156.590,1.08303,1.08310,1.08318,1.08321,1.08362,1.08347,1.08424,1.08280,1218.510639,522.69,0.0640,0.000612,1590.452174
2024-05-22 12:25:00,12.700965,49.678457,-156158.542164,37.377951,0.000435,-0.000840,-1.478629e-04,-0.000693,28.091639,1.08306,0.66500,82.013,1.36712,156.568,1.08320,1.08323,1.08362,1.08345,1.08318,1.08438,1.08461,1.08313,1.08307,0.66503,82.093,1.36740,156.599,1.08328,1.08336,1.08369,1.08369,1.08347,1.08441,1.08469,1.08319,1.08270,0.66477,81.932,1.36700,156.567,1.08305,1.08313,1.08321,1.08338,1.08314,1.08420,1.08437,1.08288,1.08280,0.66484,81.958,1.36735,156.589,1.08310,1.08318,1.08321,1.08362,1.08347,1.08424,1.08438,1.08303,962.500836,498.06,0.0698,0.000466,1041.411729
2024-05-22 12:30:00,10.945274,51.243781,-155536.965676,42.777951,0.000424,-0.000692,1.584083e-07,-0.000693,29.614699,1.08313,0.66474,82.028,1.36730,156.574,1.08323,1.08362,1.08345,1.08318,1.08438,1.08461,1.08452,1.08320,1.08319,0.66507,82.063,1.36735,156.578,1.08336,1.08369,1.08369,1.08347,1.08441,1.08469,1.08464,1.08328,1.08288,0.66464,81.927,1.36702,156.554,1.08313,1.08321,1.08338,1.08314,1.08420,1.08437,1.08446,1.08305,1.08303,0.66499,82.003,1.36712,156.567,1.08318,1.08321,1.08362,1.08347,1.08424,1.08438,1.08462,1.08310,1014.151113,519.75,0.0827,0.000454,1030.841746


In [14]:
df_joined_val.sort_index(ascending=True, inplace=True)
df_joined_val.tail()

,+DI,-DI,ADL,ADX,ATR_14,MACD,MACD_hist,MACD_signal,RSI,close,close_audusd,close_brentusd,close_cadusd,close_jpyusd,close_plus_10min,close_plus_15min,close_plus_20min,close_plus_25min,close_plus_30min,close_plus_35min,close_plus_40min,close_plus_5min,high,high_audusd,high_brentusd,high_cadusd,high_jpyusd,high_plus_10min,high_plus_15min,high_plus_20min,high_plus_25min,high_plus_30min,high_plus_35min,high_plus_40min,high_plus_5min,low,low_audusd,low_brentusd,low_cadusd,low_jpyusd,low_plus_10min,low_plus_15min,low_plus_20min,low_plus_25min,low_plus_30min,low_plus_35min,low_plus_40min,low_plus_5min,open,open_audusd,open_brentusd,open_cadusd,open_jpyusd,open_plus_10min,open_plus_15min,open_plus_20min,open_plus_25min,open_plus_30min,open_plus_35min,open_plus_40min,open_plus_5min,volume,volume_audusd,volume_brentusd,volume_cadusd,volume_jpyusd
time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-04-24 20:55:00,36.777583,22.241681,-249163.867000,65.871880,0.000492,0.000171,-0.000305,0.000476,50.640708,1.13854,0.64112,66.553,1.38589,142.707,1.13502,1.13492,1.13510,1.13487,1.13467,1.13385,1.13314,1.13503,1.13944,0.64130,66.593,1.38589,142.707,1.13507,1.13519,1.13520,1.13600,1.13511,1.13470,1.13388,1.13520,1.13853,0.64104,66.548,1.38500,142.564,1.13492,1.13484,1.13482,1.13463,1.13454,1.13384,1.13293,1.13484,1.13943,0.64121,66.588,1.38500,142.564,1.13502,1.13501,1.13490,1.13511,1.13486,1.13466,1.13381,1.13484,1284.770514,344.320000,2.500000e-09,456.27,713.880000
2025-04-25 03:00:00,16.433566,57.808858,-249163.866973,66.317079,0.000760,-0.000861,-0.001070,0.000209,25.393561,1.13503,0.64022,66.860,1.38683,142.933,1.13492,1.13510,1.13487,1.13467,1.13385,1.13314,1.13333,1.13502,1.13520,0.64035,66.860,1.38686,142.980,1.13519,1.13520,1.13600,1.13511,1.13470,1.13388,1.13343,1.13507,1.13484,0.63990,66.730,1.38672,142.924,1.13484,1.13482,1.13463,1.13454,1.13384,1.13293,1.13292,1.13492,1.13484,0.63990,66.735,1.38682,142.972,1.13501,1.13490,1.13511,1.13486,1.13466,1.13381,1.13315,1.13502,0.000486,0.000806,4.900008e-03,403.38,0.001162
2025-04-25 03:05:00,15.284360,58.767773,-249163.866829,66.963925,0.000709,-0.001335,-0.001235,-0.000100,25.354274,1.13502,0.64048,66.890,1.38676,142.926,1.13510,1.13487,1.13467,1.13385,1.13314,1.13333,1.13292,1.13492,1.13507,0.64060,66.920,1.38685,142.960,1.13520,1.13600,1.13511,1.13470,1.13388,1.13343,1.13341,1.13519,1.13492,0.64010,66.795,1.38670,142.924,1.13482,1.13463,1.13454,1.13384,1.13293,1.13292,1.13284,1.13484,1.13502,0.64010,66.820,1.38682,142.940,1.13490,1.13511,1.13486,1.13466,1.13381,1.13315,1.13333,1.13501,0.000432,0.000482,4.300008e-03,284.70,0.000804
2025-04-25 03:10:00,12.887828,59.188544,-249163.867029,67.611910,0.000679,-0.001517,-0.001134,-0.000383,24.933458,1.13492,0.64059,66.905,1.38668,142.954,1.13487,1.13467,1.13385,1.13314,1.13333,1.13292,1.13306,1.13510,1.13519,0.64071,66.925,1.38685,142.961,1.13600,1.13511,1.13470,1.13388,1.13343,1.13341,1.13336,1.13520,1.13484,0.64025,66.840,1.38666,142.909,1.13463,1.13454,1.13384,1.13293,1.13292,1.13284,1.13265,1.13482,1.13501,0.64039,66.850,1.38675,142.929,1.13511,1.13486,1.13466,1.13381,1.13315,1.13333,1.13292,1.13490,0.000368,0.000445,2.000004e-03,205.68,0.000912
2025-04-25 03:15:00,12.949640,57.074341,-249163.866773,68.709957,0.000654,-0.001473,-0.000871,-0.000601,27.302760,1.13510,0.64082,66.815,1.38660,142.920,1.13467,1.13385,1.13314,1.13333,1.13292,1.13306,1.13264,1.13487,1.13520,0.64091,66.915,1.38673,142.969,1.13511,1.13470,1.13388,1.13343,1.13341,1.13336,1.13326,1.13600,1.13482,0.64042,66.790,1.38648,142.901,1.13454,1.13384,1.13293,1.13292,1.13284,1.13265,1.13254,1.13463,1.13490,0.64049,66.910,1.38669,142.951,1.13486,1.13466,1.13381,1.13315,1.13333,1.13292,1.13305,1.13511,0.000539,0.000507,2.900005e-03,363.27,0.000993


In [15]:
print(len(df_joined_train), len(df_joined_test), len(df_joined_val))

13845 4615 4616


In [16]:
# predict_min_stats(df_joined_train, df_joined_test, df_joined_val, scaler_x_dir=SCALER_X_MIN, scaler_y_dir=SCALER_Y_MIN, model_dir=LSTM_MIN_VAL)

In [17]:
# predict_max_stats(df_joined_train, df_joined_test, df_joined_val, scaler_x_dir=SCALER_X_MAX, scaler_y_dir=SCALER_Y_MAX, model_dir=LSTM_MAX_VAL)